In [19]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [20]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Sunnyshore Bay")

In [21]:
# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

In [22]:
# The objective function
question_2_model.setObjective(1.0275*B[0]+1.0225*B[1]+1.0175*B[2]+1.0225*B[3]+1.0175*B[4]+1.0175*B[5], GRB.MINIMIZE)

In [23]:
# Add the balance constraints
question_2_model.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [24]:
# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model.addConstr(w[1] >= 20000, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

In [25]:
# Add the borrowing constraints
question_2_model.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model.addConstr(B[5] <= 350000, "July Borrowing Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [26]:
# Ratio constraint
question_2_model.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [27]:
# Optimally solve the problem
question_2_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0xea444b4c
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05


In [28]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model.objVal, 2))

Total Amount of Money:  142904.73


In [29]:
# Print the decision variables
print(question_2_model.printAttr('X'))


    Variable            X 
-------------------------
   Borrow[0]         5000 
   Borrow[3]      80945.9 
   Borrow[4]      54054.1 
   Wealth[0]        25000 
   Wealth[1]        20000 
   Wealth[2]        35000 
   Wealth[3]       327095 
None
